In [1]:
""" İşlenmiş veri setini yükleme ve kontrol """
import polars as pl
from pathlib import Path

# optimizasyon sonrası parquet verinin yolu
PROCESSED_DATA_PATH = Path("../data/processed/metropt3_optimized.parquet")

# Parqueti oku
df = pl.read_parquet(PROCESSED_DATA_PATH)

# Temel bilgiler
print(f"Satır sayısı : {df.shape[0]:,}")
print(f"Sütun sayısı : {df.shape[1]}")
print(f"Başlangıç   : {df['timestamp'].min()}")
print(f"Bitiş       : {df['timestamp'].max()}")
print(f"Süre        : {df['timestamp'].max() - df['timestamp'].min()}")

Satır sayısı : 1,516,948
Sütun sayısı : 16
Başlangıç   : 2020-02-01 00:00:00
Bitiş       : 2020-09-01 03:59:50
Süre        : 213 days, 3:59:50


Parqueti rame tam yükledik. artık lazy moda gerek yok baya küçülttük saten. Veriler temiz 1.5 milyon satır, 7 aylık veri, 16 sensör.

In [2]:
""" Zaman aralığı kontrolü """
# Timestamp farkları
time_diffs = df['timestamp'].diff().drop_nulls()
#.diff ardışık zaman damgası farkını hesaplar. .drop_nulls() ilk kayıtta oluşan null değeri temizler.

print("Zaman aralığı istatistikleri:")
print(f"Min  : {time_diffs.min()}") 
print(f"Max  : {time_diffs.max()}") 
print(f"Ortalama : {time_diffs.mean()}")
print(f"En sık görülen aralık : {time_diffs.mode()[0]}")

# Beklenmedik büyük boşluklar — 60 saniyeden fazla atlayan kayıtlar
gaps = df.filter(time_diffs.append(pl.Series([None])).shift(1) > pl.duration(seconds=60))
print(f"\n60 saniyeden uzun boşluk sayısı: {len(gaps)}")

Zaman aralığı istatistikleri:
Min  : 0:00:08
Max  : 2 days, 0:01:58
Ortalama : 0:00:12.141221
En sık görülen aralık : 0:00:10

60 saniyeden uzun boşluk sayısı: 331


Ortalama 10sn de bir veri akışı görmemiz lazım ancak 331 tane incelememiz gereken ciddi kopmalar var en önemlisi 2 günlük en büyük kopma var. Mantıksal yürütmeler ve tüm sensörlerdeki durumların o anlardaki halleri ile karşılaştırmalar sonucunda genel bakım mı, teknik onarım mı, birimsel arıza mı tespitleri yapıcaz

In [8]:
# Boşlukları büyükten küçüğe sırala ve göster
time_diffs_series = df['timestamp'].diff()

gaps_df = df.with_columns(time_diffs_series.alias('gap') #timestamp farklarını gap sütunu olarak ekle
).filter(pl.col('gap') > pl.duration(seconds=60) # gap değeri 60 saniyeden büyük olanları filtrele
).select(['timestamp', 'gap'] # sadece timestamp ve gap sütunlarını seç
).sort('gap', descending=True) # gap sütununa göre büyükten küçüğe sırala uzun kesintiler üstte

print(gaps_df.head(20))

shape: (20, 2)
┌─────────────────────┬────────────────┐
│ timestamp           ┆ gap            │
│ ---                 ┆ ---            │
│ datetime[μs]        ┆ duration[μs]   │
╞═════════════════════╪════════════════╡
│ 2020-04-27 01:12:49 ┆ 2d 1m 58s      │
│ 2020-06-28 23:07:43 ┆ 1d 12h 14m 36s │
│ 2020-03-01 04:00:09 ┆ 1d 4h 3m 1s    │
│ 2020-08-05 08:23:01 ┆ 1d 40m 33s     │
│ 2020-05-25 01:14:14 ┆ 1d 34m 51s     │
│ …                   ┆ …              │
│ 2020-03-30 07:33:30 ┆ 11h 44m 16s    │
│ 2020-03-20 06:25:41 ┆ 9h 18m 55s     │
│ 2020-05-01 21:26:05 ┆ 8h 47m 39s     │
│ 2020-06-02 18:47:39 ┆ 8h 47m 34s     │
│ 2020-04-30 23:34:46 ┆ 8h 28m 46s     │
└─────────────────────┴────────────────┘
